In [1]:
%matplotlib inline
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import os


In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model


In [3]:
train_files = pd.read_csv(r'D:\smartinternz\Ship Classification\input\train.csv')

In [4]:
train_labels = pd.read_csv(r'D:\smartinternz\Ship Classification\input\train.csv')

In [5]:
train_labels['category'] = train_labels['category'] - 1

In [6]:
train_labels.head()

,image,category
0,2823080.jpg,0
1,2870024.jpg,0
2,2662125.jpg,1
3,2900420.jpg,2
4,2804883.jpg,1


In [7]:
train_files.head()

,image,category
0,2823080.jpg,1
1,2870024.jpg,1
2,2662125.jpg,2
3,2900420.jpg,3
4,2804883.jpg,2


In [8]:
ship = {1 :'Cargo' , 
        2:'Military', 
        3:'Carrier', 
        4:'Cruise', 
        5:'Tankers'}

train_files['ship'] = train_files['category'].map(ship).astype('category')

train_files.head()

,image,category,ship
0,2823080.jpg,1,Cargo
1,2870024.jpg,1,Cargo
2,2662125.jpg,2,Military
3,2900420.jpg,3,Carrier
4,2804883.jpg,2,Military


In [10]:
labels = train_files.sort_values('ship')
class_names = list(labels.ship.unique())
for i in class_names:
    os.makedirs(os.path.join('D:/smartinternz/Ship Classification/input/train',i))


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:/smartinternz/Ship Classification/input/train\\Cargo'

In [11]:
import shutil
for c in class_names : 
    for i in list(labels[labels['ship']==c]['image']):
        get_image = os.path.join('D:/smartinternz/Ship Classification/input/images',i)
        move_image_to_cat = shutil.move(get_image, "D:/smartinternz/Ship Classification/input/train"+c)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/smartinternz/Ship Classification/input/images\\2823080.jpg'

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input

In [13]:
train_datagen = ImageDataGenerator(rotation_range=45,
                                  horizontal_flip=True,
                                  width_shift_range=0.5,
                                  height_shift_range=0.5,
                                  validation_split=0.2,
                                  preprocessing_function=preprocess_input
                                  )
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [14]:
train_set = train_datagen.flow_from_directory('D:/smartinternz/Ship Classification/input/train/',
                                              batch_size=16, subset='training',
                                              target_size=(224,224))
validation_set = train_datagen.flow_from_directory('D:/smartinternz/Ship Classification/input/train/',
                                                   batch_size=16, subset='validation',
                                                   target_size=(224,224))
test_set = test_datagen.flow_from_directory('D:/smartinternz/Ship Classification/input/test/',batch_size=16,
                                            target_size=(224,224))

Found 5003 images belonging to 5 classes.
Found 1249 images belonging to 5 classes.
Found 0 images belonging to 0 classes.


In [15]:
from keras.layers import Dense,Flatten, Dropout
from keras.models import Model

In [16]:
def create_model(input_shape,n_classes, optimizer='rmsprop'):
    conv_base= VGG16(include_top = False,
                     weights='imagenet',
                     input_shape=input_shape)
    for layer in conv_base.layers:
        layer.trainable = False 
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    model = Model(inputs=conv_base.input, outputs=output_layer)
    model.compile (optimizer = optimizer,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model
        

In [17]:
from tensorflow.keras.optimizers import Adam

In [18]:
input_shape = (224,224,3)
optim = Adam(learning_rate=0.001)
weights = 'imagenet'
n_classes=5
vgg_model = create_model(input_shape, n_classes, optim)

In [19]:
vgg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [20]:
from keras.callbacks import ModelCheckpoint
cp = ModelCheckpoint('best.hdf5', monitor = 'val_loss',verbose=1, save_best_only=True)

In [21]:
epochs = 25
history = vgg_model.fit_generator(generator=train_set,
                                 steps_per_epoch=train_set.n//train_set.batch_size,
                                 validation_steps = validation_set.n//validation_set.batch_size,
                                 validation_data=validation_set,
                                 callbacks=[cp],
                                 epochs=epochs)

C:\Users\KAREEM\AppData\Local\Temp\ipykernel_19596\1604804611.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = vgg_model.fit_generator(generator=train_set,


Epoch 1/25
312/312 [==============================] - ETA: 0s - loss: 6.8804 - accuracy: 0.5266
Epoch 1: val_loss improved from inf to 0.97463, saving model to best.hdf5
312/312 [==============================] - 911s 3s/step - loss: 6.8804 - accuracy: 0.5266 - val_loss: 0.9746 - val_accuracy: 0.6354
Epoch 2/25
312/312 [==============================] - ETA: 0s - loss: 0.8876 - accuracy: 0.6671
Epoch 2: val_loss improved from 0.97463 to 0.89312, saving model to best.hdf5
312/312 [==============================] - 899s 3s/step - loss: 0.8876 - accuracy: 0.6671 - val_loss: 0.8931 - val_accuracy: 0.6562
Epoch 3/25
312/312 [==============================] - ETA: 0s - loss: 0.8434 - accuracy: 0.6854
Epoch 3: val_loss did not improve from 0.89312
312/312 [==============================] - 895s 3s/step - loss: 0.8434 - accuracy: 0.6854 - val_loss: 0.9696 - val_accuracy: 0.6859
Epoch 4/25
312/312 [==============================] - ETA: 0s - loss: 0.8062 - accuracy: 0.6916
Epoch 4: val_loss imp

In [22]:
vgg_model.save('vgg16-ship-classification.h5')

In [31]:
from tensorflow.keras.preprocessing import image

img= image.load_img('D:/smartinternz/Ship Classification/input/test/Cargo/Cargo1.jpg',target_size=(224,224))
img = image.img_to_array(img)
img = img.reshape((1,img.shape[0], img.shape[1], img.shape[2]))
img = preprocess_input(img)

pred = vgg_model.predict(img)
pred = pred.flatten()
pred= list(pred)
m = max(pred)

val_dict = {0:'Cargo', 1:'Carrier', 2:'Cruise', 3:'Military', 4:'Tankers'}
result=val_dict[pred.index(m)]
print(result)

1/1 [==============================] - 0s 493ms/step
Cargo
